# 使用SVM预测donor位点
> 生科登峰1901班，吴思承
>
> U201912536

## 概述
真核生物的基因包含外显子和内含子，在转录完成后，需要进行加工，选择性地将外显子连接起来，最终得到成熟的mRNA。而研究显示，在内含子与外显子的边界上，存在保守的剪接位点。其中，我们将位于内含子左侧（5'端）的剪接位点称为供体（donor），而位于内含子右侧（3'端）的剪接位点称为受体（acceptor）。

在基因预测工作中，研究人员往往需要明确DNA中实际编码蛋白质的部分。此时便需要预测RNA上的选择性剪接方式，而作为内含子边界上的保守序列，donor位点的识别对内含子、外显子的位置、数量的确定有着巨大的用处。

预测donor位点的方法包括WMM、WAM到SVN、神经网络等，颇为丰富。本上机报告选择实现其中的SVM模型，并对其实际预测效果进行评估。

## 方法

### SVM的原理
包括LSVM在内的线性分类器在训练时试图在样本空间内寻找一个超平面，将训练数据阳性和阴性样本区分开。根据新样本与超平面的位置关系，进行分类判别。但是这一方法仅适用于样本本身线性可分的情况，若样本线性不可分，则传统的线性分类器难以得到好的结果。

SVM将数据样本视为特征空间中的点（或向量），使用核函数，将相对低维的点映射到更高维空间中，使得原本线性不可分的样本变得线性可分。之后再在高维特征空间中寻找超平面，实现分类。

### 依赖加载与数据挂载
与先前的WAM上机任务类似，在本次上机中，本人同样使用Python（Jupyter Notebook）实现SVM模型。代码在Google Colab的云服务上运行，数据存储在账号对应的Google Drive中。

调用的依赖库中，``os``用于读取文件，``re``用于正则表达式辨别数据中的外显子信息，``random``提供随机采样函数，``matplotlib``用于绘制图像，``tqdm``用于生成数据读取的进度条，``sklearn``用于提供现成的模型性能评估方法和SVM实现。

In [1]:
import os
import re
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, f1_score, confusion_matrix

# 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 数据储存路径
training_path = '/content/drive/MyDrive/Collab Files/donor_dataset/training'
testing_path = '/content/drive/MyDrive/Collab Files/donor_dataset/testing'


Mounted at /content/drive


### 从数据集中提取序列片段
与WAM不同，SVM不仅仅通过统计donor位点附近的碱基分布来构建打分表，而是需要同时使用阳性样本和阴性样本的序列片段来构建判别器。因此需要对训练数据中的各种序列片段进行提取。

下面的``get_sequence``函数重复使用了WAM上机任务中的代码。

In [2]:
# get_sequence：从数据集中提取需要的序列片段
def get_sequence(file_path, k1=4, k2=4, type='positive'):
    sequence_list = []

    # 逐个读取文件
    print('Handling files in \'' + file_path + '\' ...')
    for data_file in tqdm(os.listdir(file_path)):
        with open(file_path + '/' + data_file, 'r') as FILE:
            lines = FILE.readlines()
            sequence = ''.join(lines[2:]).replace('\n', '').lower()
            donor_sites = re.findall('(\d+)(?=,)', lines[1])

            # 输出阳性样本
            if type == 'positive':
                for site in donor_sites:
                    site_num = eval(site)
                    subsequence = sequence[site_num - k1 - 1:site_num + k2]

                    if (set(subsequence) | {'a', 't', 'c', 'g'}) != {'a', 't', 'c', 'g'}:
                        continue

                    sequence_list.append(subsequence)

            # 输出阴性样本
            else:
                window_size = k1 + k2 + 1
                seq_len = len(sequence)

                for i in range(seq_len - window_size + 1):
                    subsequence = sequence[i:i + window_size]

                    if (set(subsequence) | {'a', 't', 'c', 'g'}) != {'a', 't', 'c', 'g'} \
                       or str(i + k1 + 1) in donor_sites:
                        continue
                    
                    sequence_list.append(subsequence)

    return sequence_list

### 对序列进行one-hot编码
由于SVM将样本序列视为高维空间中的一个点（向量），用0~3的数字来表示某一位置上的碱基将会引入原始数据中不存在的位置关系，可能会对模型产生负面影响。因此，有必要对碱基进行one-hot编码（独热编码），将离散的碱基映射到欧氏空间中，并确保不同碱基之间的距离相同，以便后续训练。

函数``one_hot_sequence``将输入的序列处理为one-hot编码的numpy数组，输出的数组长度将会是输入序列长度的4倍。

碱基到one-hot编码的映射关系储存在字典``nucleotide_to_array``中。与WAM中的处理方法类似，表示不确定碱基的``n``、``b``等将会被直接忽略。

In [3]:
# nucleotide_to_array：提供从碱基到one-hot编码的映射关系
nucleotide_to_array = {
    'a': np.array([1, 0, 0, 0], dtype=float),
    't': np.array([0, 1, 0, 0], dtype=float),
    'c': np.array([0, 0, 1, 0], dtype=float),
    'g': np.array([0, 0, 0, 1], dtype=float)
}

print('nucleotide_to_array: ', nucleotide_to_array)

nucleotide_to_array:  {'a': array([1., 0., 0., 0.]), 't': array([0., 1., 0., 0.]), 'c': array([0., 0., 1., 0.]), 'g': array([0., 0., 0., 1.])}


In [4]:
# one_hot_sequence：将序列处理为one-hot编码的数组
def one_hot_sequence(sequence, nucleotide_to_array=nucleotide_to_array):
    one_hot_list = [nucleotide_to_array[nt] for nt in sequence]
    one_hot_array = np.array(one_hot_list).reshape(-1)
    return one_hot_array

### 构建数据集
结合前面的两个函数，按照需要构建one-hot编码完毕的数据集，用于训练和验证。

函数``build_dataset``返回两个列表``data_list``和``data_class``，分别储存编码后的样本序列和样本的类型（是否是donor位点附近序列）。输入中的``pos_num``和``neg_num``为数据集中阳性样本和阴性样本的数量，为默认值``None``或大于样本总数时输出全部样本。

In [5]:
# build_dataset：构建数据集
def build_dataset(file_path, pos_num=None, neg_num=None, k1=4, k2=4):
    # 从训练用文件中提取所有的序列
    positive_seq = get_sequence(file_path, k1, k2, 'positive')
    negative_seq = get_sequence(file_path, k1, k2, 'negative')

    # 随机选取pos_num个阳性样本
    if pos_num != None and pos_num < len(positive_seq):
        index = random.sample(range(len(positive_seq)), pos_num)
        positive_list = [one_hot_sequence(positive_seq[i]) for i in index]
    else:
        positive_list = [one_hot_sequence(seq) for seq in positive_seq]
    
    # 随机选取neg_num个阴性样本
    if neg_num != None and neg_num < len(negative_seq):
        index = random.sample(range(len(negative_seq)), neg_num)
        negative_list = [one_hot_sequence(negative_seq[i]) for i in index]
    else:
        negative_list = [one_hot_sequence(seq) for seq in negative_seq]

    # 合并成训练集
    data_list = positive_list + negative_list
    data_class = [1] * len(positive_list) + [0] * len(negative_list)

    return data_list, data_class

### 构建SVM模型
使用现有的``sklearn.svm.SVC``构建SVM模型，并进行训练。

函数``build_svm``输入训练集和核函数类型，返回训练完毕的SVM分类器。其中核函数参数``kernel``默认为``'rbf'``，即高斯核（径向基函数核）。考虑到训练数据中阳性和阴性样本数量可能极其不均衡，参数``class_weight``设置为``'balanced'``，根据类别比例自动设置惩罚参数。``sklearn.svm.SVC``的其余参数均使用默认值。

In [6]:
# build_svm：构建模型并训练，返回训练完毕的模型
def build_svm(train_x, train_y, kernel='rbf'):
    classifier = SVC(kernel=kernel, class_weight='balanced')
    classifier.fit(train_x, train_y)
    return classifier

### 滑动窗口识别序列中的donor位点
对于一段给定的序列，使用滑动窗口法，对序列中的每一个位点周围的片段进行SVM分类，但由于SVM无法辨别经验上的高度保守。该部分代码也与WAM的类似。

使用函数``search_donor_site``实现这一功能。参数中``k1``、``k2``与``nucleotide_frequency_count``中相同，为位点前后纳入判别的碱基数量。函数输出一个列表，其中包含了可能的donor位点的位置，下标从1开始。

In [7]:
# search_donor_site：查找序列中的donor位点
def search_donor_site(sequence, classifier, k1=4, k2=4):
    donor_site = []
    window_size = k1 + k2 + 1
    seq_len = len(sequence)

    # 滑动窗口判断donor位点
    for i in range(seq_len - window_size + 1):
        subsequence = sequence[i:i + window_size]
        
        # 跳过包含不确定碱基的片段
        if (set(subsequence) | {'a', 't', 'c', 'g'}) != {'a', 't', 'c', 'g'}:
            continue

        # 将SVM判断为donor位点，且符合经验的位置加入结果列表
        subseq_array = one_hot_sequence(subsequence).reshape(1, -1)
        if classifier.predict(subseq_array)[0] == 1 and subsequence[5:7] == 'gt':
            donor_site.append(i + k1 + 1)
    
    return donor_site

## 结果
### 模型训练
使用训练集中的序列构建训练集，并分别训练使用线性核函数的SVM和使用高斯核的SVM。命名为``classifier_linear``和``classifier_rbf``。

由于阴性样本数量过大，将其数量限制在100000。相对而言，使用高斯核的SVM需要更长的时间训练。

In [8]:
# 构建训练集
train_list, train_class = build_dataset(training_path, neg_num=100000)

# 训练SVM
classifier_linear = build_svm(train_list, train_class, kernel='linear')
classifier_rbf = build_svm(train_list, train_class, kernel='rbf')

Handling files in '/content/drive/MyDrive/Collab Files/donor_dataset/training' ...


100%|██████████| 462/462 [00:03<00:00, 119.02it/s]


Handling files in '/content/drive/MyDrive/Collab Files/donor_dataset/training' ...


100%|██████████| 462/462 [00:10<00:00, 42.09it/s]


### 进行预测
从验证集中选取一个文件为例。分别使用线性核与高斯核通过``search_donor_site``函数对序列中的donor位点进行预测，以确认算法能否正常运行。

可以看出，虽然依旧存在一定量的假阳性，但是高斯核有着优于线性核的结果。

In [9]:
# 选取一个文件，从中得到序列
file_path = testing_path + '/ACU08131.TXT'

with open(file_path, 'r') as FILE:
    lines = FILE.readlines()
    sequence = ''.join(lines[2:]).replace('\n', '').lower()
    
    # 提取出实际的donor位点
    donor_sites = re.findall('(\d+)(?=,)', lines[1])
    
    # 预测donor位点
    predict_donor_linear = search_donor_site(sequence, classifier_linear)
    predict_donor_rbf = search_donor_site(sequence, classifier_rbf)

    print('Actural donor sites: \n[', ', '.join(donor_sites) + ']', sep='')
    print('Predicted donor sites (by \'classifier_linear\'): \n', predict_donor_linear, sep='')
    print('Predicted donor sites (by \'classifier_rbf\'): \n', predict_donor_rbf, sep='')

Actural donor sites: 
[641, 1362, 2028, 2802, 3797]
Predicted donor sites (by 'classifier_linear'): 
[12, 69, 115, 135, 157, 264, 295, 321, 325, 363, 386, 462, 501, 532, 550, 641, 709, 749, 789, 812, 823, 844, 848, 860, 868, 879, 881, 924, 982, 993, 999, 1008, 1023, 1077, 1142, 1172, 1178, 1229, 1254, 1262, 1327, 1362, 1419, 1436, 1440, 1522, 1602, 1653, 1734, 1824, 1859, 1885, 1933, 1957, 1981, 2028, 2034, 2039, 2078, 2133, 2178, 2211, 2244, 2259, 2349, 2387, 2437, 2456, 2497, 2502, 2536, 2546, 2597, 2636, 2676, 2689, 2701, 2781, 2797, 2802, 2806, 2822, 2839, 2877, 2985, 2989, 3010, 3041, 3223, 3335, 3404, 3426, 3513, 3557, 3608, 3620, 3623, 3779, 3797, 3817, 3824, 3832, 3870, 3896, 3898, 3906, 3913, 3981, 3983, 4037, 4134, 4166, 4208, 4305, 4451, 4503, 4511, 4562, 4589, 4686, 4718, 4865, 4919, 4970, 4991, 5025, 5169, 5175, 5285, 5312, 5323, 5344, 5360, 5375]
Predicted donor sites (by 'classifier_rbf'): 
[12, 69, 115, 157, 321, 325, 386, 462, 501, 532, 641, 709, 789, 812, 844, 881, 92

## 讨论
从验证集中提取阳性donor样本和阴性donor样本，并使用高斯核SVM进行预测，用于后续分析。

与训练集一样，运行时间起见，阴性样本的数量被控制在100000条。

In [10]:
test_list, test_class = build_dataset(testing_path, neg_num=100000)
test_pred = classifier_rbf.predict(test_list)

Handling files in '/content/drive/MyDrive/Collab Files/donor_dataset/testing' ...


100%|██████████| 570/570 [00:04<00:00, 118.66it/s]


Handling files in '/content/drive/MyDrive/Collab Files/donor_dataset/testing' ...


100%|██████████| 570/570 [00:06<00:00, 91.74it/s] 


### F1-score与混淆矩阵
分别计算F1-score和混淆矩阵。

混淆矩阵的格式依旧是
```
[[TN    FP]
 [FN    TP]]
```

计算得F1-score为0.721。可以看出，SVM得到了比前一报告中的WAM更高的F1-score，且假阳性率更低。


In [11]:
f1 = f1_score(test_class, test_pred)
conf_matrix = confusion_matrix(test_class, test_pred)

print('F1-score: ', f1)
print('Confusion Matrix: ')
print(conf_matrix)

F1-score:  0.7176983435047951
Confusion Matrix: 
[[98402  1598]
 [   21  2058]]


### 结论
虽然需要更长的训练时间，且偶尔会出现一些不符合碱基保守性的结果。但相较于WAM，使用高斯核的SVM获得了相对更好的donor位点识别性能。

不过显然，SVM在实际应用时的假阳性结果依旧不让人满意，有巨大的改进空间。